In [20]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from pipeline import create_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [21]:
import time

In [22]:
# labeling
# Get Dataset
files={
    'CICI':'/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedCICI.csv',
    'UNSW': '/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedUNSW.csv'
}

data = pd.read_csv(files['CICI'])
data=data[np.isfinite(data).all(1)]
multiclass_labels=data['new_category']

data=data.drop(labels=['label','attack_category','nist_category','new_category','Unnamed: 0'],axis=1)

In [23]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [24]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','recall_mi','recall_ma','recall_we']+\
                 ['precision_mi','precision_ma','precision_we'])
eval_path='/home/irteam/junghye-dcloud-dir/MLAC/evaluation'

confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/new_cici'
if os.path.isdir(confusion_path)==False:
    os.mkdir(confusion_path)
cnt=0

In [25]:
X_train,X_test,y_train,y_test=train_test_split(data,multiclass_labels,test_size=0.3, shuffle=True, stratify=multiclass_labels, random_state=34)


In [26]:
import matplotlib.pyplot as plt

In [27]:
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue',pad_inches=0.5)
    plt.clf()

In [ ]:
for name, model in models:

    model=create_pipeline(model)
    print('training start...'+name)
    start_time=time.time()
    model.fit(X_train,y_train)
    end_time=time.time()
    
    print("모델 training 소요 시간: {:.2f}초".format(end_time - start_time))


    #evaluation
    
    print('evaluation start...')
    start_time=time.time()
    y_pred=model.predict(X_test)
    end_time=time.time()
    print("모델 test 소요 시간: {:.2f}초".format(end_time - start_time))

    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, y_pred)
    f1_mi = f1_score(y_test, y_pred,average='micro')
    f1_ma = f1_score(y_test, y_pred,average='macro')
    f1_we = f1_score(y_test, y_pred,average='weighted')
    recall_mi = recall_score(y_test, y_pred, average='micro')
    recall_ma = recall_score(y_test, y_pred, average='macro')
    recall_we = recall_score(y_test, y_pred, average='weighted')
    precision_mi = precision_score(y_test, y_pred, average='micro')
    precision_ma = precision_score(y_test, y_pred, average='macro')
    precision_we = precision_score(y_test, y_pred, average='weighted')
    
    model_eval.append(acc)
    model_eval.append(f1_mi)
    model_eval.append(f1_ma)
    model_eval.append(f1_we)
    model_eval.append(recall_mi)
    model_eval.append(recall_ma)
    model_eval.append(recall_we)
    model_eval.append(precision_mi)
    model_eval.append(precision_ma)
    model_eval.append(precision_we)
    
    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,y_pred)
    plot_confusion_matrix(confusion,labels=['Benign','DoS','Reconnaissance']+\
                          ['Brute Force','Malware','Web Attack','Heartbleed'],title=name)
       


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'new_cici.csv'),index=False)

training start...RF
모델 training 소요 시간: 10.17초
evaluation start...
모델 test 소요 시간: 1.19초
name:RF,acc:0.9531017473261598,f1_score:0.9531017473261598,0.4067373195489277,0.947781573471819,recall:0.9531017473261598,0.3991409033727163,0.9531017473261598,precision:0.9531017473261598,0.4166742113606772,0.9460582865338634
training start...CART
모델 training 소요 시간: 37.05초
evaluation start...
모델 test 소요 시간: 0.90초
name:CART,acc:0.96991978093132,f1_score:0.96991978093132,0.512077097067673,0.9660345116530735,recall:0.96991978093132,0.5629781570667648,0.96991978093132,precision:0.96991978093132,0.48519240148534476,0.9624046334326705
training start...NB
모델 training 소요 시간: 5.23초
evaluation start...
모델 test 소요 시간: 3.42초
name:NB,acc:0.7156805021766242,f1_score:0.7156805021766243,0.47965381111271144,0.7806124078380259,recall:0.7156805021766242,0.8143615337086125,0.7156805021766242,precision:0.7156805021766242,0.5376146500345634,0.9061538693860438
training start...LDA
모델 training 소요 시간: 16.92초
evaluation star